# LeadPulse – End-to-End PySpark Delta Lake Pipeline

This notebook demonstrates:
- Bronze → Silver → Gold architecture
- Delta Change Data Feed (CDF)
- SCD Type 1 & Type 2
- Late-arriving data handling
- SLA breach detection
- Ranking using window functions

All transformations are implemented using PySpark and Delta Lake.


In [1]:
# %pip uninstall pyspark -y
# %pip install pyspark==3.5.1
# %pip install delta-spark


In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark.version


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/15 13:09:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'4.1.1'

In [3]:
spark.sparkContext._jvm.scala.util.Properties.versionString()

'version 2.13.17'

In [4]:
BRONZE_PATH = "data/bronze/delta_bronze_cdf"

bronze_df = spark.read.format("delta").load(BRONZE_PATH)

bronze_df.show(5)


Py4JJavaError: An error occurred while calling o28.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Make sure the provider name is correct and the package is properly registered and compatible with your Spark version. SQLSTATE: 42K02
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:764)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:686)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:745)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource$$anonfun$apply$1.applyOrElse(ResolveDataSource.scala:58)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource$$anonfun$apply$1.applyOrElse(ResolveDataSource.scala:45)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsUpWithPruning$3(AnalysisHelper.scala:139)
	at org.apache.spark.sql